In [1]:
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
synthese_df = pd.read_csv('../data/raw/Agribalyse_Synthese.csv')
raw_ingred_df = pd.read_csv('../data/raw/Agribalyse_Detail ingredient.csv')
ingred_df = pd.read_csv('../data/interim/Agribalyse_Detail ingredient.csv')
etapes_df = pd.read_csv('../data/interim/Agribalyse_Detail etape.csv')

In [7]:
for c in ingred_df.columns:
    print(c)


Code_AGB
Code_CIQUAL
Groupe_aliment
Sous-groupe_aliment
Nom_Produit_Francais
LCI_Name
Saisonnalite
Transport_par_avion_
Livraison
Materiau_emballage
Preparation
DQR_Note_qualite_la_donnee_
Score_unique_EF_
Ciqual_AGB
Abats_boeuf
Abats_porc
Abats_poulet
Abats_veau
Abricot
Ail
Amande
Amidon_mais
Ananas
Anchois
Anchois_commun
Asperges
Aubergine
Autres_etapes
Avocat
Avoine
Baie
Basilic
Beurre
Biere
Blanc_oeuf
Ble_dur
Boisson_a_base_avoine
Boisson_a_base_soja
Bouillon
Brocoli
Boeuf_hache
Cacahuete
Cacao
Cafe
Calamar
Camembert
Cannelle_
Carotte
Cassonade
Cerises
Champignon
Chicoree
Chicoree_rouge
Chocolat_lait
Chocolat_noir
Chou_blanc
Chou_vert
Choucroute
Choufleur
Chataigne
Ciboulette_ou_oignon_printemps
Clous_girofle
Comte
Concentre_tomates_30deg_Brix
Concombre
Congre
Coquille_Saint-Jacques
Coriandre
Courgette
Cresson_jardin
Crevette
Creme
Celeri
Celeri-rave
Eau_Vie
Emmental
Endive
Endive_hiver
Epinard
Escargot
Estragon
Farine_ble
Farine_riz
Farine_sarrasin
Farine_seigle
Fenouil
Fond_veau_

In [10]:
synthese_df[synthese_df['Nom du Produit en Français'].str.contains('Brocoli')]

,Code AGB,Code CIQUAL,Groupe d'aliment,Sous-groupe d'aliment,Nom du Produit en Français,LCI Name,Saisonnalité,Transport par avion (1 : par avion),Livraison,Matériau d'emballage,...,Particules (E-06 disease inc./kg de produit),Acidification terrestre et eaux douces (mol H+ eq/kg de produit),Eutrophisation terreste (mol N eq/kg de produit),Eutrophisation eaux douces (E-03 kg P eq/kg de produit),Eutrophisation marine (E-03 kg N eq/kg de produit),Utilisation du sol (Pt/kg de produit),Écotoxicité pour écosystèmes aquatiques d'eau douce (CTUe/kg de produit),Épuisement des ressources eau (m3 depriv./kg de produit),Épuisement des ressources énergétiques (MJ/kg de produit),Épuisement des ressources minéraux (E-06 kg Sb eq/kg de produit)
341,20057,20057,"fruits, légumes, légumineuses et oléagineux",légumes,"Brocoli, cru","Broccoli, raw",mix de consommation FR,0,Ambiant (moyenne),Pas d'emballage,...,0.033102,0.004065,0.014009,0.119384,4.955335,8.830993,17.059999,0.997938,12.319274,2.412127
342,20006,20006,"fruits, légumes, légumineuses et oléagineux",légumes,"Brocoli, cuit","Broccoli, cooked",mix de consommation FR,0,Glacé,PP,...,0.052241,0.006401,0.020608,0.213697,6.200879,11.054100,24.643907,1.314691,33.886057,3.881295
343,20259,20259,"fruits, légumes, légumineuses et oléagineux",légumes,"Brocoli, purée","Broccoli, puree",mix de consommation FR,0,Glacé,PP,...,0.051550,0.006278,0.020358,0.207403,6.177475,10.991912,24.015092,1.275232,31.873316,3.816935
344,20204,20204,"fruits, légumes, légumineuses et oléagineux",légumes,"Brocoli, surgelé, cru","Broccoli, frozen, raw",mix de consommation FR,0,Congelé,LPDE,...,0.050037,0.006109,0.018871,0.231518,5.401721,10.620305,23.329960,1.342738,37.682095,3.727172
345,20205,20205,"fruits, légumes, légumineuses et oléagineux",légumes,"Brocoli, surgelé, cuit","Broccoli, frozen, cooked",mix de consommation FR,0,Congelé,LPDE,...,0.059655,0.007314,0.022401,0.281746,6.345648,12.505745,28.279945,1.623685,47.641929,4.486718


### Etapes

In [30]:
drop_cols = etapes_df.columns[[1, 6, 7, 11]].tolist()
print(drop_cols)
etapes_df = etapes_df.drop(columns=drop_cols)
etapes_df[etapes_df['Code_AGB'] == '20057']

['Groupe_aliment', 'Materiau_emballage', 'Preparation', 'Score_unique_EF__Emballage']


,Code_AGB,Sous-groupe_aliment,Nom_Produit_Francais,LCI_Name,Livraison,Score_unique_EF_,Score_unique_EF__Agriculture,Score_unique_EF__Transformation,Score_unique_EF__Transport,Score_unique_EF__Supermarche_distribution,Score_unique_EF__Consommation
341,20057,légumes,"Brocoli, cru","Broccoli, raw",Ambiant (moyenne),0.082754,0.055642,0.0,0.010936,0.015654,0.000522


### Ingredients

In [35]:
raw_ingred_df

,Ciqual AGB,Groupe d'aliment,Sous-groupe d'aliment,LCI Name,Changement climatique (kg CO2 eq/kg de produit),Particules (E-06 disease inc./kg de produit),Acidification terrestre et eaux douces (mol H+ eq/kg de produit),Eutrophisation terreste (mol N eq/kg de produit),Eutrophisation marine (E-03 kg N eq/kg de produit),Utilisation du sol (Pt/kg de produit),Écotoxicité pour écosystèmes aquatiques d'eau douce (CTUe/kg de produit),Épuisement des ressources eau (m3 depriv./kg de produit),Épuisement des ressources énergétiques (MJ/kg de produit),Épuisement des ressources minéraux (E-06 kg Sb eq/kg de produit)
0,11168,aides culinaires et ingrédients divers,sauces,"Aioli sauce (garlic and olive oil mayonnaise),...",0.164754,0.046616,0.006761,0.029441,1.329230,16.984964,7.404112,0.337769,1.475548,0.251356
1,11168,aides culinaires et ingrédients divers,sauces,"Aioli sauce (garlic and olive oil mayonnaise),...",0.011444,0.000781,0.000105,0.000328,0.189983,1.643787,1.216336,0.291209,0.229937,0.084028
2,11168,aides culinaires et ingrédients divers,sauces,"Aioli sauce (garlic and olive oil mayonnaise),...",0.061230,0.006365,0.000951,0.003573,0.213721,3.707242,2.625043,1.445978,0.793865,0.203959
3,11168,aides culinaires et ingrédients divers,sauces,"Aioli sauce (garlic and olive oil mayonnaise),...",1.161825,0.142041,0.021524,0.088459,9.263219,7.579606,464.100950,9.374618,12.100965,4.424012
4,11168,aides culinaires et ingrédients divers,sauces,"Aioli sauce (garlic and olive oil mayonnaise),...",0.571172,0.024611,0.002630,0.008368,1.295442,4.529954,11.433712,0.759522,22.626111,1.278583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5666,19552,lait et produits laitiers,produits laitiers frais et assimilés,"Yogurt, Greek-style, on a bed of fruits",0.113532,0.027982,0.000952,0.003176,0.370286,3.561618,195.859930,1.702002,1.587775,3.330941
5667,19552,lait et produits laitiers,produits laitiers frais et assimilés,"Yogurt, Greek-style, on a bed of fruits",0.549462,0.023034,0.002426,0.007428,1.159934,4.096425,6.238759,0.248905,21.610336,1.176044
5668,19860,lait et produits laitiers,produits laitiers frais et assimilés,"Yogurt, Greek-style, plain",0.693950,0.043025,0.006846,0.029968,8.579385,62.085522,1.178242,0.013914,1.062461,0.019234
5669,19860,lait et produits laitiers,produits laitiers frais et assimilés,"Yogurt, Greek-style, plain",1.198262,0.089744,0.013406,0.058485,3.903060,66.972761,17.562407,0.373698,3.073724,0.642726


In [36]:
drop_cols = raw_ingred_df.columns[[1, 6, 7, 11]].tolist()
print(drop_cols)
#raw_ingred_df = raw_ingred_df.drop(columns=drop_cols)
raw_ingred_df[raw_ingred_df['Ciqual AGB'] == '20057']

["Groupe d'aliment", 'Acidification terrestre et eaux douces (mol H+ eq/kg de produit)', 'Eutrophisation terreste (mol N eq/kg de produit)', 'Épuisement des ressources eau (m3 depriv./kg de produit)']


,Ciqual AGB,Groupe d'aliment,Sous-groupe d'aliment,LCI Name,Changement climatique (kg CO2 eq/kg de produit),Particules (E-06 disease inc./kg de produit),Acidification terrestre et eaux douces (mol H+ eq/kg de produit),Eutrophisation terreste (mol N eq/kg de produit),Eutrophisation marine (E-03 kg N eq/kg de produit),Utilisation du sol (Pt/kg de produit),Écotoxicité pour écosystèmes aquatiques d'eau douce (CTUe/kg de produit),Épuisement des ressources eau (m3 depriv./kg de produit),Épuisement des ressources énergétiques (MJ/kg de produit),Épuisement des ressources minéraux (E-06 kg Sb eq/kg de produit)


In [24]:
drop_cols = synthese_df.columns[[1, 6, 7, 11]].tolist()
print(drop_cols)
synthese_df = synthese_df.drop(columns=drop_cols, axis=1)
synthese_df[synthese_df['Nom du Produit en Français'].str.contains('cru')]


['Code CIQUAL', 'Saisonnalité', 'Transport par avion (1 : par avion)', 'DQR - Note de qualité de la donnée (1 excellente ; 5 très faible)']


,Code AGB,Groupe d'aliment,Sous-groupe d'aliment,Nom du Produit en Français,LCI Name,Livraison,Matériau d'emballage,Préparation,Score unique EF (mPt/kg de produit),Changement climatique (kg CO2 eq/kg de produit),...,Particules (E-06 disease inc./kg de produit),Acidification terrestre et eaux douces (mol H+ eq/kg de produit),Eutrophisation terreste (mol N eq/kg de produit),Eutrophisation eaux douces (E-03 kg P eq/kg de produit),Eutrophisation marine (E-03 kg N eq/kg de produit),Utilisation du sol (Pt/kg de produit),Écotoxicité pour écosystèmes aquatiques d'eau douce (CTUe/kg de produit),Épuisement des ressources eau (m3 depriv./kg de produit),Épuisement des ressources énergétiques (MJ/kg de produit),Épuisement des ressources minéraux (E-06 kg Sb eq/kg de produit)
4,13000,"fruits, légumes, légumineuses et oléagineux",fruits,"Abricot, dénoyauté, cru","Apricot, pitted, raw",Ambiant (moyenne),LPDE,Pas de préparation,0.118929,0.861730,...,0.061462,0.007254,0.025954,0.204244,0.943669,43.838644,41.957928,1.412878,17.123820,2.173328
7,11084,aides culinaires et ingrédients divers,algues,"Agar (algue), cru","Seaweed, agar, raw",Ambiant (long),LPDE,Pas de préparation,1.252574,6.547801,...,0.701238,0.098439,0.116177,2.240995,11.705482,21.750919,157.987250,4.925442,341.429410,30.229683
9,21514,"viandes, œufs, poissons",viandes crues,"Agneau, collier, cru","Lamb, neck, raw",Glacé,PS,Pas de préparation,4.792772,49.578452,...,5.041112,0.748324,3.322237,2.516765,157.151610,4381.337200,378.678060,12.203794,92.026149,17.502130
10,21516,"viandes, œufs, poissons",viandes crues,"Agneau, côte filet, crue","Lamb, chop fillet, raw",Glacé,PS,Pas de préparation,4.792772,49.578452,...,5.041112,0.748324,3.322237,2.516765,157.151610,4381.337200,378.678060,12.203794,92.026149,17.502130
12,21517,"viandes, œufs, poissons",viandes crues,"Agneau, côte première, crue","Lamb, rib chop, raw",Glacé,PS,Pas de préparation,4.792772,49.578452,...,5.041112,0.748324,3.322237,2.516765,157.151610,4381.337200,378.678060,12.203794,92.026149,17.502130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2431,6540,"viandes, œufs, poissons",viandes crues,"Veau, poitrine, crue","Veal, breast, raw",Glacé,PS,Pas de préparation,1.432853,15.976774,...,1.003944,0.150006,0.641831,1.414224,99.911337,973.125520,200.512510,4.290596,69.878943,8.250447
2432,6550,"viandes, œufs, poissons",viandes crues,"Veau, rôti, cru","Veal, roast, raw",Glacé,PS,Pas de préparation,2.232983,24.926367,...,1.566749,0.234407,1.003891,2.220842,156.140800,1520.226500,315.614560,6.644251,105.856190,12.631246
2434,6536,"viandes, œufs, poissons",viandes crues,"Veau, steak haché 15% MG, cru","Veal, minced steak, 15% fat, raw",Glacé,PS,Pas de préparation,2.212514,24.377724,...,1.536243,0.229833,0.983845,2.191516,153.181850,1492.338200,311.508520,6.629972,115.450920,12.640988
2435,6535,"viandes, œufs, poissons",viandes crues,"Veau, steak haché 20% MG, cru","Veal, minced steak, 20% fat, raw",Glacé,PS,Pas de préparation,2.212514,24.377724,...,1.536243,0.229833,0.983845,2.191516,153.181850,1492.338200,311.508520,6.629972,115.450920,12.640988


In [ ]:
21514

In [17]:
synth_ing_diff=set(synthese_df['Code CIQUAL'])- set(raw_ingred_df['Ciqual AGB'])
synth_ing_diff

In [4]:
synthese_df[synthese_df['Code CIQUAL'].isin(list(synth_ing_diff))].value_counts()

NameError: name 'synth_ing_diff' is not defined

In [32]:
#synthese_df[synthese_df['Sous-groupe d\'aliment'] == 'Oeuf']
raw_ingred_df[raw_ingred_df['Ingredients'].isin(['Oeuf'])]

,Ciqual AGB,Ciqual code,Nom Français,Groupe d'aliment,Sous-groupe d'aliment,LCI Name,Ingredients,Score unique EF (mPt/kg de produit),Changement climatique (kg CO2 eq/kg de produit),Appauvrissement de la couche d'ozone (E-06 kg CVC11 eq/kg de produit),...,Particules (E-06 disease inc./kg de produit),Acidification terrestre et eaux douces (mol H+ eq/kg de produit),Eutrophisation terreste (mol N eq/kg de produit),Eutrophisation eaux douces (E-03 kg P eq/kg de produit),Eutrophisation marine (E-03 kg N eq/kg de produit),Utilisation du sol (Pt/kg de produit),Écotoxicité pour écosystèmes aquatiques d'eau douce (CTUe/kg de produit),Épuisement des ressources eau (m3 depriv./kg de produit),Épuisement des ressources énergétiques (MJ/kg de produit),Épuisement des ressources minéraux (E-06 kg Sb eq/kg de produit)


In [13]:
raw_ingred_df[raw_ingred_df['Ingredients'] == 'Lait écrémé']

,Ciqual AGB,Ciqual code,Nom Français,Groupe d'aliment,Sous-groupe d'aliment,LCI Name,Ingredients,Score unique EF (mPt/kg de produit),Changement climatique (kg CO2 eq/kg de produit),Appauvrissement de la couche d'ozone (E-06 kg CVC11 eq/kg de produit),...,Particules (E-06 disease inc./kg de produit),Acidification terrestre et eaux douces (mol H+ eq/kg de produit),Eutrophisation terreste (mol N eq/kg de produit),Eutrophisation eaux douces (E-03 kg P eq/kg de produit),Eutrophisation marine (E-03 kg N eq/kg de produit),Utilisation du sol (Pt/kg de produit),Écotoxicité pour écosystèmes aquatiques d'eau douce (CTUe/kg de produit),Épuisement des ressources eau (m3 depriv./kg de produit),Épuisement des ressources énergétiques (MJ/kg de produit),Épuisement des ressources minéraux (E-06 kg Sb eq/kg de produit)
2091,19659,19659,"Fromage blanc ou spécialité laitière, aux frui...",lait et produits laitiers,produits laitiers frais et assimilés,"Fresh cream cheese, with fruits, creamy, with ...",Lait écrémé,0.090364,1.017585,0.02696,...,0.076212,0.011385,0.049667,0.091451,3.314548,56.87445,14.914306,0.317351,2.610261,0.545815


In [6]:
for c in ingred_df.columns:
    print(c)


Code_AGB
Code_CIQUAL
Groupe_aliment
Sous-groupe_aliment
Nom_Produit_Francais
LCI_Name
Saisonnalite
Transport_par_avion_
Livraison
Materiau_emballage
Preparation
DQR_Note_qualite_la_donnee_
Score_unique_EF_
Ciqual_AGB
Abats de bœuf
Abats de porc
Abats de poulet
Abats de veau
Abricot
Ail
Amande
Amidon de maïs
Ananas
Anchois
Anchois commun
Asperges
Aubergine
Autres étapes
Avocat
Avoine
Baie
Basilic
Beurre
Bière
Blanc d'oeuf
Blé dur
Boisson à base d'avoine
Boisson à base de soja
Bouillon
Brocoli
Bœuf haché
Cacahuète
Cacao
Café
Calamar
Camembert
Cannelle 
Carotte
Cassonade
Cerises
Champignon
Chicorée
Chicorée rouge
Chocolat au lait
Chocolat noir
Chou blanc
Chou vert
Choucroute
Choufleur
Châtaigne
Ciboulette ou oignon de printemps
Clous de girofle
Comté
Concentré de tomates 30° Brix
Concombre
Congre
Coquille Saint-Jacques
Coriandre
Courgette
Cresson de jardin
Crevette
Crème
Céleri
Céleri-rave
Eau de Vie
Emmental
Endive
Endive d'hiver
Epinard
Escargot
Estragon
Farine de blé
Farine de riz
Fari

In [8]:
def missing(df):
    missing = ingred_df.isna().sum()
    missing[missing>0]

In [71]:
missing(ingred_df)
ingred_df.fillna(0, inplace=True)


In [9]:
missing(ingred_df)
ingred_df['Livraison'].value_counts()

Glacé                1353
Ambiant (moyenne)     482
Ambiant (long)        475
Congelé                96
Ambiant (court)        73
Name: Livraison, dtype: int64

In [73]:
only_ingred = ingred_df.iloc[:, 15:]
only_ingred

,Abats de bœuf,Abats de porc,Abats de poulet,Abats de veau,Abricot,Ail,Amande,Amidon de maïs,Ananas,Anchois,...,Viande de moutton sans os,Viande de porc maigre,Viande de poulet sans os,Viande de veau sans os,Vin blanc,Vin rouge,Yaourt,citron,Échalote,Œuf de poule
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.046785
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.114290,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.109798,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.109798,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1035,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1036,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [77]:
only_ingred[only_ingred['Abats de porc']>0]
for i,c in enumerate(only_ingred.columns.to_list()):
    if i == 1:
        only_ingred.loc[only_ingred[only_ingred[c] > 0].index, c] = 1
        print(c)
    break

In [78]:
only_ingred[only_ingred['Abats de bœuf'] > 0]

,Abats de bœuf,Abats de porc,Abats de poulet,Abats de veau,Abricot,Ail,Amande,Amidon de maïs,Ananas,Anchois,...,Viande de moutton sans os,Viande de porc maigre,Viande de poulet sans os,Viande de veau sans os,Vin blanc,Vin rouge,Yaourt,citron,Échalote,Œuf de poule
5,0.463192,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
6,0.463192,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
7,0.463192,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
8,0.463192,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
9,0.463192,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
10,0.463192,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
11,0.463192,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
139,0.463192,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
140,0.707618,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
141,0.707618,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
